In [ ]:
import docx
import json
import pandas as pd
import nltk

import random
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [ ]:
# функция для генерации
correct_addresses = []
republics = ['Чувашская', 'Чеченская', 'Кабардино-Балкарская']
names_rep = ['республика', 'область']
city = ['Москва', 'Санкт-Петебург', 'Чебоксары']
street_name = ['Моховая', 'Тверская', 'Невская']
def generate_address():
    for i in range(1000):
        new_str = random.choice(republics) + ' ' + random.choice(names_rep) + ', город ' + random.choice(city) + ', улица ' + random.choice(street_name)
    
    correct_addresses.append(new_str)


In [ ]:
# функция для скрытия слов
def masking(s, word):
    new_s = s.replace(word, '[MASK]')
    return new_s

In [ ]:
# нейросеть для обработки
for original in zip(correct_addresses):
    print(f"Original: {original}")
    print()


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize(addresses):
    return tokenizer(addresses, padding=True, truncation=True, return_tensors="pt")

inputs = tokenize([masking(i, 'Республика') for i in correct_addresses])
labels = tokenize(correct_addresses)


class AddressCorrector(nn.Module):
    def __init__(self):
        super(AddressCorrector, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=tokenizer.vocab_size)
    
    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs

model = AddressCorrector()
optimizer = AdamW(model.parameters(), lr=5e-5)

dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels['input_ids'])


dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

def train(model, dataloader, optimizer, epochs=3):
    model.train()
    for epoch in range(epochs):
        for batch in dataloader:
            input_ids, attention_mask, label_ids = batch
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=label_ids)
            loss = outputs.loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f"Epoch {epoch}, Loss: {loss.item()}")

train(model, dataloader, optimizer)

def predict(model, address):
    model.eval()
    inputs = tokenize([address])
    with torch.no_grad():
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    predicted_ids = torch.argmax(outputs.logits, dim=-1)
    return tokenizer.decode(predicted_ids[0])

test_address = "Чувашская ресублика, город Чебоксары, улица Моховая"
corrected_address = predict(model, test_address)
print(f"Original: {test_address}")
print(f"Corrected: {corrected_address}")


In [ ]:
# второй вариант обработки, приведен для примера

string = '428023 Чувашская Република, город Чебоксары, бульвар Миттова, '
words = string.split()
for i in range(len(words)):
    new_word = words[i].strip(',')
    dist = nltk.edit_distance('Республика', new_word)
    if 1 <= dist <= 3:
        if words[i][-1] == ',':
            words[i] = 'Республика' + ','
        else:
            words[i] = 'Республика'

' '.join(words)